In [1]:
%pip install --upgrade opencv-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install mediapipe


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyautogui


Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyautogui
print(pyautogui.size())  # Gets screen resolution


Size(width=1920, height=1080)


In [5]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install handtrackingmodule

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Set up webcam
cap = cv2.VideoCapture(0)

# Variables to calculate FPS
pTime = 0

# Screen dimensions
screen_width, screen_height = pyautogui.size()

# Gesture control variables
gesture_threshold = 50  # Distance threshold for gesture detection
finger_states = [False] * 5  # State of each finger (open or closed)

# Main loop
while True:
    success, img = cap.read()
    if not success:
        break

    # Flip the image horizontally for natural interaction
    img = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Process the image to detect hands
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extracting landmarks
            landmarks = hand_landmarks.landmark

            # Calculate the bounding box of the hand
            h, w, _ = img.shape
            x_min, y_min = w, h
            x_max, y_max = 0, 0

            for id, lm in enumerate(landmarks):
                cx, cy = int(lm.x * w), int(lm.y * h)
                x_min, y_min = min(x_min, cx), min(y_min, cy)
                x_max, y_max = max(x_max, cx), max(y_max, cy)

                # Mark each landmark with a circle (optional for debugging)
                cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

            # Check if index finger is up
            if landmarks[8].y < landmarks[6].y:  # Tip above PIP joint
                finger_states[1] = True
            else:
                finger_states[1] = False

            # Check if thumb is up
            if landmarks[4].x < landmarks[3].x:  # Tip left of IP joint (for right hand)
                finger_states[0] = True
            else:
                finger_states[0] = False

            # Move mouse pointer with index finger
            if finger_states[1] and not any(finger_states[2:]):  # Only index finger up
                # Map hand coordinates to screen coordinates
                x = np.interp(landmarks[8].x, [0, 1], [0, screen_width])
                y = np.interp(landmarks[8].y, [0, 1], [0, screen_height])
                pyautogui.moveTo(x, y)

            # Left click gesture (thumb and index finger together)
            if finger_states[0] and finger_states[1]:
                thumb_tip = landmarks[4]
                index_tip = landmarks[8]
                distance = ((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2) ** 0.5
                if distance < gesture_threshold:  # Check if tips are close
                    pyautogui.click()
                    time.sleep(0.2)  # Avoid multiple clicks

            # Right click gesture (thumb and pinky finger together)
            if finger_states[0] and landmarks[20].x < landmarks[19].x:  # Pinky near thumb
                pyautogui.rightClick()
                time.sleep(0.2)

    # Calculate FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (10, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

    # Show the image with hand landmarks and FPS
    cv2.imshow("Hand Tracking", img)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
